In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("Timeline")

---

Table of Contents <a class="jp-toc-ignore"></a>
=================
* [1 Data preparation](#data-preparation)
  * [1.1 Load data](#load-data)
  * [1.2 Merge data](#merge-data)
* [2 Data visualization](#data-visualization)
  * [2.1 Debut](#debut)
    * [2.1.1 By Format](#by-format)
    * [2.1.2 By Region](#by-region)
    * [2.1.3 By Card type](#by-card-type)
    * [2.1.4 By Primary type](#by-primary-type)
    * [2.1.5 By Secondary type](#by-secondary-type)
    * [2.1.6 By Attribute](#by-attribute)
    * [2.1.7 By Monster type](#by-monster-type)
    * [2.1.8 By Level/Rank](#by-level/rank)
    * [2.1.9 By Pendulum scale](#by-pendulum-scale)
    * [2.1.10 By Link](#by-link)
    * [2.1.11 By ATK](#by-atk)
    * [2.1.12 By DEF](#by-def)
  * [2.2 Last release](#last-release)
    * [2.2.1 By Region](#by-region)
    * [2.2.2 By Card type](#by-card-type)
    * [2.2.3 By Primary type](#by-primary-type)
    * [2.2.4 By Secondary type](#by-secondary-type)
    * [2.2.5 By Attribute](#by-attribute)
    * [2.2.6 By Monster type](#by-monster-type)
    * [2.2.7 By Level/Rank](#by-level/rank)
    * [2.2.8 By Pendulum scale](#by-pendulum-scale)
    * [2.2.9 By Link](#by-link)
    * [2.2.10 By ATK](#by-atk)
    * [2.2.11 By DEF](#by-def)
  * [2.3 All releases](#all-releases)
    * [2.3.1 By Region](#by-region)
    * [2.3.2 By Card type](#by-card-type)
    * [2.3.3 By Primary type](#by-primary-type)
    * [2.3.4 By Secondary type](#by-secondary-type)
    * [2.3.5 By Attribute](#by-attribute)
    * [2.3.6 By Monster type](#by-monster-type)
    * [2.3.7 By Level/Rank](#by-level/rank)
    * [2.3.8 By Pendulum scale](#by-pendulum-scale)
    * [2.3.9 By Link](#by-link)
    * [2.3.10 By ATK](#by-atk)
    * [2.3.11 By DEF](#by-def)
* [3 Merge failed](#merge-failed)
  * [3.1 Names missing](#names-missing)
  * [3.2 Card number missing](#card-number-missing)
* [4 Epilogue](#epilogue)
  * [4.1 HTML export](#html-export)
  * [4.2 Git](#git)

# Data preparation

In [ ]:
timestamp = arrow.utcnow()

## Load data

In [ ]:
# Load list of important dates
dates_json = load_json(dirs.get_asset("json", "dates.json"))
anime_df = pd.DataFrame(dates_json["anime"]["series"]).set_index("title").map(pd.to_datetime, dayfirst=True)
rules_df = (
    pd.DataFrame(dates_json["rules"]).set_index("title").map(pd.to_datetime, dayfirst=True).iloc[2:]
)  # Ignore old rules

In [ ]:
# Get latest file if exist
cards_df = load_latest_data("cards")
speed_df = load_latest_data("speed")
set_lists_df = load_latest_data("sets")

## Merge data

In [ ]:
df_list = [cards_df, speed_df, set_lists_df]
if any(item is None for item in df_list):
    raise SystemExit("Not enough files to proceed. Aborting!")

In [ ]:
set_lists_df = set_lists_df[
    ~set_lists_df["Card number"].str.startswith("RD/").astype(bool).fillna(False)
]  # Remove Rush duel cards
full_df = merge_set_to_cards(cards_df, speed_df, set_df=set_lists_df)

# Data visualization

Rush duel and Bandai cards are not included.

In [ ]:
full_df

## Debut

In [ ]:
debut_counts = get_releases_by(full_df, column=None, operation="debut").value_counts(sort=False).rename("All formats")
debut_counts.groupby(debut_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(debut_counts, bg=anime_df, vlines=rules_df["begin"], limit_year=True)
plt.show()

### By Format

Monsters' debut per format

In [ ]:
full_df.groupby("Name")[full_df.filter(regex="(?i)(debut)").columns].min()

Debuts per date for each format

In [ ]:
format_debut_counts = (
    full_df.groupby("Name")[full_df.filter(regex="(?i)(debut)").columns]
    .min()
    .melt(var_name="Format", value_name="Debut")
    .value_counts()
    .unstack(0)
    .fillna(0)
    .sort_index()
)
format_debut_counts.groupby(format_debut_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(format_debut_counts, bg=anime_df, vlines=rules_df["begin"], subplots=True)
plt.show()

### By Region

Obs: Debut by region is taken from earliest release date in set lists. It may not be as accurate as card specific properties.

In [ ]:
full_df.groupby(["Region", "Name"])["Release"].min().unstack(0)

First releases per date

In [ ]:
region_debut_counts = get_releases_by(full_df, column="Region", operation="first", crosstab=True)
region_debut_counts.groupby(region_debut_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(
    region_debut_counts,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=True,
    limit_year=True,
)
plt.show()

### By Card type

In [ ]:
ct_debut_counts = get_releases_by(full_df, column="Card type", operation="debut", crosstab=True)
ct_debut_counts.groupby(ct_debut_counts.index.strftime("%Y")).sum()

In [ ]:
_ct_debut_plot = ct_debut_counts[["Monster Card", "Spell Card", "Trap Card", "Skill Card"]]
_ct_colors = [plot.colors_dict[col] for col in _ct_debut_plot.columns]
_ = plot.rate(
    _ct_debut_plot,
    colors=_ct_colors,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=False,
    limit_year=True,
)

### By Primary type

In [ ]:
pt_debut_counts = get_releases_by(full_df, column="Primary type", operation="debut", crosstab=True)
pt_debut_counts.groupby(pt_debut_counts.index.strftime("%Y")).sum()

In [ ]:
_pt_colors = [plot.colors_dict[col] for col in pt_debut_counts.columns]
_ = plot.rate(
    pt_debut_counts,
    colors=_pt_colors,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=False,
    limit_year=True,
)

### By Secondary type

In [ ]:
st_debut_counts = get_releases_by(full_df, "Secondary type", operation="debut", crosstab=True)
st_debut_counts.groupby(st_debut_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(
    st_debut_counts,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=True,
    limit_year=True,
)

### By Attribute

In [ ]:
att_debut_counts = get_releases_by(full_df, column="Attribute", operation="debut", crosstab=True)
att_debut_counts.groupby(att_debut_counts.index.strftime("%Y")).sum()

In [ ]:
attribute_colors = [plot.colors_dict[col] for col in att_debut_counts.columns]
_ = plot.rate(
    att_debut_counts,
    colors=attribute_colors,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=False,
    limit_year=True,
)

### By Monster type

In [ ]:
mt_debut_counts = get_releases_by(full_df, column="Monster type", operation="debut", crosstab=True)
mt_debut_counts.groupby(mt_debut_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(mt_debut_counts, bg=anime_df, vlines=rules_df["begin"], subplots=True, limit_year=True)

### By Level/Rank

In [ ]:
level_debut = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card") & (full_df["Primary type"] != "Link Monster")],
    column="Level/Rank/Link",
    operation="debut",
    crosstab=False,
)
level_debut.groupby(level_debut["Debut"].dt.year)["Level/Rank/Link"].describe()

In [ ]:
_ = plot.box(
    level_debut,
    color=plot.colors_dict["Effect Monster"],
    notch=True,
)
plt.show()

### By Pendulum scale

In [ ]:
pendulum_debut = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card")], column="Pendulum Scale", operation="debut", crosstab=False
)
pendulum_debut.groupby(pendulum_debut["Debut"].dt.year)["Pendulum Scale"].describe()

In [ ]:
_ = plot.box(
    pendulum_debut,
    color=plot.colors_dict["Spell Card"],
    notch=True,
)
plt.show()

### By Link

In [ ]:
link_debut = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card") & (full_df["Primary type"] == "Link Monster")],
    column="Level/Rank/Link",
    operation="debut",
    crosstab=False,
)
link_debut.groupby(link_debut["Debut"].dt.year)["Level/Rank/Link"].describe()

In [ ]:
_ = plot.box(
    link_debut,
    color=plot.colors_dict["Link Monster"],
    notch=True,
)
plt.show()

### By ATK

In [ ]:
atk_debut = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card")], column="ATK", operation="debut", crosstab=False
)
atk_debut.groupby(atk_debut["Debut"].dt.year)["ATK"].describe()

In [ ]:
_ = plot.box(
    atk_debut,
    color=plot.colors_dict["Effect Monster"],
    notch=True,
)
plt.show()

### By DEF

In [ ]:
def_debut = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card")], column="DEF", operation="debut", crosstab=False
)
def_debut.groupby(def_debut["Debut"].dt.year)["DEF"].describe()

In [ ]:
_ = plot.box(
    def_debut,
    color=plot.colors_dict["Effect Monster"],
    notch=True,
)
plt.show()

## Last release

Obs: Only the last release of an individual card name

In [ ]:
last_counts = get_releases_by(full_df, column=None, operation="last").value_counts(sort=False).rename("All formats")
last_counts.groupby(last_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(last_counts, bg=anime_df, vlines=rules_df["begin"], limit_year=True)
plt.show()

### By Region

In [ ]:
full_df.groupby(["Region", "Name"])["Release"].max().unstack(0)

Last releases by date

In [ ]:
region_last_counts = get_releases_by(full_df, column="Region", operation="last", crosstab=True)
region_last_counts.groupby(region_last_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(region_last_counts, bg=anime_df, vlines=rules_df["begin"], subplots=True)
plt.show()

### By Card type

In [ ]:
ct_last_counts = get_releases_by(full_df, column="Card type", operation="last", crosstab=True)
ct_last_counts.groupby(ct_last_counts.index.strftime("%Y")).sum()

In [ ]:
_ct_last_plot = ct_last_counts[["Monster Card", "Spell Card", "Trap Card", "Skill Card"]]
_ct_colors = [plot.colors_dict[col] for col in _ct_last_plot.columns]
_ = plot.rate(
    _ct_last_plot,
    colors=_ct_colors,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=False,
    limit_year=True,
)

### By Primary type

In [ ]:
pt_last_counts = get_releases_by(full_df, column="Primary type", operation="last", crosstab=True)
pt_last_counts.groupby(pt_last_counts.index.strftime("%Y")).sum()

In [ ]:
_pt_colors = [plot.colors_dict[col] for col in pt_last_counts.columns]
_ = plot.rate(
    pt_last_counts,
    colors=_pt_colors,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=False,
    limit_year=True,
)

### By Secondary type

In [ ]:
st_last_counts = get_releases_by(full_df, column="Secondary type", operation="last", crosstab=True)
st_last_counts.groupby(st_last_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(
    st_last_counts,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=True,
    limit_year=True,
)

### By Attribute

In [ ]:
att_last_counts = get_releases_by(full_df, column="Attribute", operation="last", crosstab=True)
att_last_counts.groupby(att_last_counts.index.strftime("%Y")).sum()

In [ ]:
attribute_colors = [plot.colors_dict[col] for col in att_last_counts.columns]
_ = plot.rate(
    att_last_counts,
    colors=attribute_colors,
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=False,
    limit_year=True,
)

### By Monster type

In [ ]:
mt_last_counts = get_releases_by(full_df, column="Monster type", operation="last", crosstab=True)
mt_last_counts.groupby(mt_last_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(
    mt_last_counts,
    title="Debut",
    bg=anime_df,
    vlines=rules_df["begin"],
    subplots=True,
    limit_year=True,
)

### By Level/Rank

In [ ]:
level_last = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card") & (full_df["Primary type"] != "Link Monster")],
    column="Level/Rank/Link",
    operation="last",
    crosstab=False,
)
level_last.groupby(level_last["Last release"].dt.year)["Level/Rank/Link"].describe()

In [ ]:
_ = plot.box(
    level_last,
    color=plot.colors_dict["Effect Monster"],
    notch=True,
)
plt.show()

### By Pendulum scale

In [ ]:
pendulum_last = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card")], column="Pendulum Scale", operation="last", crosstab=False
)
pendulum_last.groupby(pendulum_last["Last release"].dt.year)["Pendulum Scale"].describe()

In [ ]:
_ = plot.box(
    pendulum_last,
    color=plot.colors_dict["Spell Card"],
    notch=True,
)
plt.show()

### By Link

In [ ]:
link_last = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card") & (full_df["Primary type"] == "Link Monster")],
    column="Level/Rank/Link",
    operation="last",
    crosstab=False,
)
link_last.groupby(link_last["Last release"].dt.year)["Level/Rank/Link"].describe()

In [ ]:
_ = plot.box(
    link_last,
    color=plot.colors_dict["Link Monster"],
    notch=True,
)
plt.show()

### By ATK

In [ ]:
atk_last = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card")], column="ATK", operation="last", crosstab=False
)
atk_last.groupby(atk_last["Last release"].dt.year)["ATK"].describe()

In [ ]:
_ = plot.box(
    atk_last,
    color=plot.colors_dict["Effect Monster"],
    notch=True,
)
plt.show()

### By DEF

In [ ]:
def_last = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card")], column="DEF", operation="last", crosstab=False
)
def_last.groupby(def_last["Last release"].dt.year)["DEF"].describe()

In [ ]:
_ = plot.box(
    def_last,
    color=plot.colors_dict["Effect Monster"],
    notch=True,
)
plt.show()

## All releases

Obs: All releases includes reprints

In [ ]:
full_df[full_df["Release"].notna()].groupby("Name")["Release"].unique().explode().reset_index().set_index(
    "Release"
).sort_index()

In [ ]:
all_releases = (
    get_releases_by(full_df, column=None, operation="all", crosstab=False).value_counts(sort=False).rename("All formats")
)
_ = plot.rate(all_releases, bg=anime_df, vlines=rules_df["begin"], limit_year=True)
plt.show()

### By Region

In [ ]:
region_release_counts = get_releases_by(full_df, column="Region", operation="all", crosstab=True)
region_release_counts.groupby(region_release_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(region_release_counts, bg=anime_df, vlines=rules_df["begin"], limit_year=True, subplots=True)
plt.show()

### By Card type

In [ ]:
ct_release_counts = get_releases_by(full_df, column="Card type", operation="all", crosstab=True)
ct_release_counts.groupby(ct_release_counts.index.strftime("%Y")).sum()

In [ ]:
_ct_release_plot = ct_release_counts[["Monster Card", "Spell Card", "Trap Card", "Skill Card"]]
_ct_colors = [plot.colors_dict[col] for col in _ct_release_plot.columns]
_ = plot.rate(_ct_release_plot, colors=_ct_colors, bg=anime_df, vlines=rules_df["begin"], limit_year=True)
plt.show()

### By Primary type

In [ ]:
pt_release_counts = get_releases_by(full_df, column="Primary type", operation="all", crosstab=True)
pt_release_counts.groupby(pt_release_counts.index.strftime("%Y")).sum()

In [ ]:
primary_type_colors = [plot.colors_dict[col] for col in pt_release_counts.columns]
_ = plot.rate(pt_release_counts, colors=primary_type_colors, bg=anime_df, vlines=rules_df["begin"], limit_year=True)
plt.show()

### By Secondary type

In [ ]:
st_release_counts = get_releases_by(full_df, column="Secondary type", operation="all", crosstab=True)
st_release_counts.groupby(st_release_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(st_release_counts, bg=anime_df, vlines=rules_df["begin"], limit_year=True, subplots=True)
plt.show()

### By Attribute

In [ ]:
att_release_counts = get_releases_by(full_df, column="Attribute", operation="all", crosstab=True)
att_release_counts.groupby(att_release_counts.index.strftime("%Y")).sum()

In [ ]:
attribute_colors = [plot.colors_dict[col] for col in att_release_counts.columns]
_ = plot.rate(
    att_release_counts, colors=attribute_colors, bg=anime_df, vlines=rules_df["begin"], cumsum=True, limit_year=True
)
plt.show()

### By Monster type

In [ ]:
mt_release_counts = get_releases_by(full_df, column="Monster type", operation="all", crosstab=True)
mt_release_counts.groupby(mt_release_counts.index.strftime("%Y")).sum()

In [ ]:
_ = plot.rate(mt_release_counts, bg=anime_df, vlines=rules_df["begin"], subplots=True, limit_year=True)
plt.show()

### By Level/Rank

In [ ]:
level_release = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card") & (full_df["Primary type"] != "Link Monster")],
    column="Level/Rank/Link",
    operation="all",
    crosstab=False,
)
level_release.groupby(level_release["All releases"].dt.year)["Level/Rank/Link"].describe()

In [ ]:
_ = plot.box(
    level_release,
    color=plot.colors_dict["Level"],
    notch=True,
)
plt.show()

### By Pendulum scale

In [ ]:
pendulum_release = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card")],
    column="Pendulum Scale",
    operation="all",
    crosstab=False,
)
pendulum_release.groupby(pendulum_release["All releases"].dt.year)["Pendulum Scale"].describe()

In [ ]:
_ = plot.box(
    pendulum_release,
    color=plot.colors_dict["Pendulum Monster"],
    notch=True,
)
plt.show()

### By Link

In [ ]:
link_release = get_releases_by(
    full_df[(full_df["Card type"] != "Non-game card") & (full_df["Primary type"] == "Link Monster")],
    column="Level/Rank/Link",
    operation="all",
    crosstab=False,
)
link_release.groupby(link_release["All releases"].dt.year)["Level/Rank/Link"].describe()

In [ ]:
_ = plot.box(
    link_release,
    color=plot.colors_dict["Link Monster"],
)
plt.show()

### By ATK

In [ ]:
atk_release = get_releases_by(
    full_df[full_df["Card type"] != "Non-game card"], column="ATK", operation="all", crosstab=False
)
atk_release.groupby(atk_release["All releases"].dt.year)["ATK"].describe()

In [ ]:
_ = plot.box(
    atk_release,
    color=plot.colors_dict["Effect Monster"],
    notch=True,
)
plt.show()

### By DEF

In [ ]:
def_release = get_releases_by(
    full_df[full_df["Card type"] != "Non-game card"], column="DEF", operation="all", crosstab=False
)
def_release.groupby(def_release["All releases"].dt.year)["DEF"].describe()

In [ ]:
_ = plot.box(
    def_release,
    color=plot.colors_dict["Effect Monster"],
    notch=True,
)
plt.show()

# Merge failed

## Names missing

Likely due to card being renamed but the old name kept in the set lists

In [ ]:
cards_df.where(~cards_df["Name"].isin(full_df["Name"])).dropna(how="all")

## Card number missing

Likely old OCG cards without card number

In [ ]:
set_lists_df.where(
    (~set_lists_df["Card number"].isin(full_df["Card number"]))
    & (~set_lists_df["Card number"].dropna().str.startswith("RD/"))
).dropna(how="all")

# Epilogue

In [ ]:
benchmark(report="timeline", timestamp=timestamp)

In [ ]:
footer()

## HTML export

In [ ]:
# May need to sleep for a few seconds after saving
save_notebook()

In [ ]:
export_notebook(dirs.NOTEBOOKS.user / "Timeline.ipynb")

## Git

In [ ]:
git.commit("*[Tt]imeline*", f"Timeline update - {timestamp.isoformat()}")